In [ ]:
"""
Given an n x n binary matrix grid, return the length of the shortest clear path in the matrix. If there is no clear path, return -1.

A clear path in a binary matrix is a path from the top-left cell (i.e., (0, 0)) to the bottom-right cell (i.e., (n - 1, n - 1)) such that:

All the visited cells of the path are 0.
All the adjacent cells of the path are 8-directionally connected (i.e., they are different and they share an edge or a corner).
The length of a clear path is the number of visited cells of this path.



Example 1:
    Input: grid = [[0,1],[1,0]]
    Output: 2

Example 2:
    Input: grid = [[0,0,0],[1,1,0],[1,1,0]]
    Output: 4

Example 3:
    Input: grid = [[1,0,0],[1,1,0],[1,1,0]]
    Output: -1
 

Constraints:
    n == grid.length
    n == grid[i].length
    1 <= n <= 100
    grid[i][j] is 0 or 1
    
TIP:

(1)
If an interviewer asks you this question in an interview, then their goal is probably to determine that:

You can recognize that this is a typical shortest path problem that can be solved with a Breadth-first search (BFS). You can correctly implement a BFS to solve it. For bonus points, you know that the solution could be optimized using the A* algorithm.

Finding the shortest path between two nodes in a graph is almost always done using BFS, and all programmers should know this. BFS is one of the fundamental algorithms that you are expected to be confident coding before a tech interview. So, if you're finding this question challenging, then you're doing the right thing by working on it now.

(2) Approach 1

BFS + Overwrite grid value of visited to 1, to keep track of visited

This approach is nice in that it's very intuitive—it's directly based on how you might solve the problem on a whiteboard. It also avoids the need for a "visited" set or data structures to keep track of distances, thus saving a constant amount of memory over typical BFS implementations. However, like all in-place algorithms, overwriting the input can cause problems. Here are a couple of possible scenarios you need to consider.

That the algorithm is running in a * multithreaded* environment, and it does not have exclusive access to the grid. Other threads might need to read the grid too, and might not expect it to be modified.

That there is only a single thread or the algorithm has exclusive access to the grid while running, but the grid might need to be reused later or by another thread once the lock has been released.

(2.a) Solution to above,
    a. overwrite with value 2, at the end, re-write grid to be 0 if 2, but can't protect against multi-threading !

(3) Approach 2

What I did -- BFS + O(N) set for tracking visited

(4) Approach 3

A* -- heuristic based search algo.
    modify the algorithm to prioritize promising paths over not so promising paths.

    To do this, we need to come up with a heuristic that, given a potential option, it measures how much "promise" that option has. Then we prioritize the options (exploring cells) with the highest "promise". In the previous approaches, we were actually doing this; our heuristic was simply distance traveled so far. But we can do better than that!

# https://leetcode.com/problems/shortest-path-in-binary-matrix/editorial/

"""
# TC = SC = O(N)
from typing import List
from collections import deque
class Solution:
    def shortestPathBinaryMatrix(self, grid: List[List[int]]) -> int:
        visited = {(0, 0)}
        nodes = deque([(0, 0, 1)])
        R, C = len(grid), len(grid[0])
        if R == 1 and C == 1:
            return 1 if grid[0][0] == 0 else -1
        if grid[0][0] == 1:
            return -1
        while nodes:
            x, y, l = nodes.pop()
            for dx, dy in [(-1, 0), (1, 0), (0, 1), (0, -1), (1, 1), (1, -1), (-1, 1), (-1, -1)]:
                p, q = x+dx, y+dy
                if (
                    p < 0 or p >= R or
                    q < 0 or q >= C or
                    (p, q) in visited or
                    grid[p][q] == 1
                ):
                    continue
                if (p, q) == (R-1, C-1):
                    return l+1
                nodes.appendleft((p, q, l+1))
                visited.add((p, q))
        return -1

# A * --- heuristic is at a given level -- more optimistic choice would be the one closest to end-node, basically with least remaining node-distance;
# TC = O(NlogN), SC = O(N)
import heapq
class Solution:
    def shortestPathBinaryMatrix(self, grid: List[List[int]]) -> int:
        
        max_row = len(grid) - 1
        max_col = len(grid[0]) - 1
        directions = [
            (-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        
        # Helper function to find the neighbors of a given cell.
        def get_neighbours(row, col):
            for row_difference, col_difference in directions:
                new_row = row + row_difference
                new_col = col + col_difference
                if not(0 <= new_row <= max_row and 0 <= new_col <= max_col):
                    continue
                if grid[new_row][new_col] != 0:
                    continue
                yield (new_row, new_col)
        
        # Helper function for the A* heuristic.
        def best_case_estimate(row, col):
            return max(max_row - row, max_col - col)
            
        # Check that the first and last cells are open. 
        if grid[0][0] or grid[max_row][max_col]:
            return -1
        
        # Set up the A* search.
        visited = set()
        # Entries on the priority queue are of the form
        # (total distance estimate, distance so far, (cell row, cell col))
        priority_queue = [(1 + best_case_estimate(0, 0), 1, (0, 0))]
        while priority_queue:
            estimate, distance, cell = heapq.heappop(priority_queue)
            if cell in visited:
                continue
            if cell == (max_row, max_col):
                return distance
            visited.add(cell)
            for neighbour in get_neighbours(*cell):
                # The check here isn't necessary for correctness, but it
                # leads to a substantial performance gain.
                if neighbour in visited:
                    continue
                estimate = best_case_estimate(*neighbour) + distance + 1
                entry = (estimate, distance + 1, neighbour)
                heapq.heappush(priority_queue, entry)
        
        # There was no path.
        return -1